---
title: "Market Overview"
---

<style>
/* CSS for the tabbed layout */
.panel-tabset {
    border-bottom: 2px solid #dee2e6;
    margin-bottom: 1.5rem;
}

.panel-tabset > .nav-tabs .nav-link {
    border: none;
    border-bottom: 2px solid transparent;
    color: #6c757d;
    font-weight: 500;
    text-transform: uppercase;
    font-size: 0.9rem;
    padding: 0.75rem 1.25rem;
    transition: color 0.2s ease, border-color 0.2s ease;
}

.panel-tabset > .nav-tabs .nav-link:hover {
    color: #000;
}

.panel-tabset > .nav-tabs .nav-link.active {
    color: #000;
    border-color: #0056b3;
    background-color: transparent;
}


/* CSS for the market dashboard grid */
.market-grid {
    display: grid;
    grid-template-columns: repeat(auto-fill, minmax(300px, 1fr));
    gap: 1.5rem;
    padding-top: 1rem;
}

.market-card {
    border: 1px solid #e0e0e0;
    border-radius: 8px;
    padding: 1.5rem;
    background-color: #fff;
    box-shadow: 0 4px 6px rgba(0,0,0,0.05);
    transition: transform 0.2s ease-in-out, box-shadow 0.2s ease-in-out;
}

.market-card:hover {
    transform: translateY(-5px);
    box-shadow: 0 8px 12px rgba(0,0,0,0.08);
}

.card-header {
    display: flex;
    justify-content: space-between;
    align-items: baseline;
    margin-bottom: 0.5rem;
}

.card-header h3 {
    margin: 0;
    font-size: 1.2rem;
    font-weight: 600;
    color: #333;
}

.card-header .price {
    font-size: 1.5rem;
    font-weight: 700;
    color: #1a1a1a;
}

.card-change {
    font-size: 1rem;
    font-weight: 500;
    margin-bottom: 1rem;
}

.positive {
    color: #28a745; /* Green */
}

.negative {
    color: #dc3545; /* Red */
}

.sparkline-chart {
    width: 100%;
    height: 80px;
}

.error-card {
    background-color: #f8d7da;
    color: #721c24;
    padding: 1rem;
    border-radius: 8px;
}
</style>

::: {.panel-tabset}

## UK

In [1]:
#| echo: false
# This python chunk generates the UK market cards using Polygon.io API

import os
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display, HTML
from polygon import RESTClient
from datetime import date, timedelta
from dotenv import load_dotenv

# --- Load API Key from .env file ---
load_dotenv() 
api_key = os.getenv("POLYGON_API_KEY")

# --- Tickers for the UK dashboard (Polygon format) ---
# Using common stocks and forex pairs to ensure they are on the free plan
uk_tickers = {
    "VOD": "VODAFONE",
    "BARC": "BARCLAYS",
    "C:GBPUSD": "GBP/USD",
    "C:GBPEUR": "GBP/EUR",
    "AZN": "ASTRAZENECA",
    "X:XAUUSD": "GOLD/USD"
}

def create_market_card(client, ticker_symbol, name):
    """ Fetches data for a given ticker from Polygon.io and returns an HTML card. """
    try:
        # --- FIX: Use named argument 'ticker' ---
        snapshot = client.get_snapshot_ticker(ticker=ticker_symbol)
        
        # Check if we have valid snapshot data
        if not snapshot or not hasattr(snapshot, 'day') or not hasattr(snapshot, 'prev_day'):
             return f"""<div class="market-card error-card">
                           <h3>{name}</h3>
                           <p>Snapshot data not found for '{ticker_symbol}'.</p>
                       </div>"""

        current_price = snapshot.day.close
        change = snapshot.day.change
        percent_change = snapshot.day.change_percent

        # --- Fetch historical data for the sparkline chart ---
        today = date.today()
        sixty_days_ago = today - timedelta(days=60)
        # --- FIX: Use named argument 'ticker' ---
        hist = client.get_aggs(ticker=ticker_symbol, multiplier=1, timespan="day", from_=sixty_days_ago.strftime("%Y-%m-%d"), to=today.strftime("%Y-%m-%d"))
        
        if not hist:
            chart_html = "<p>No historical data for sparkline.</p>"
        else:
            hist_df = pd.DataFrame(hist)
            hist_df['timestamp'] = pd.to_datetime(hist_df['timestamp'], unit='ms')
            change_color_class = "positive" if change >= 0 else "negative"
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=hist_df['timestamp'], y=hist_df['close'], mode='lines', line=dict(color=change_color_class, width=2), fill='tozeroy', fillcolor=f'rgba({ "40, 167, 69" if change >= 0 else "220, 53, 69" }, 0.1)'))
            fig.update_layout(showlegend=False, margin=dict(l=0, r=0, t=0, b=0), xaxis=dict(visible=False), yaxis=dict(visible=False), plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', height=80)
            chart_html = fig.to_html(full_html=False, include_plotlyjs=False, config={'displayModeBar': False})
        
        # --- Format data for display ---
        price_str = f"{current_price:,.4f}" if "C:" in ticker_symbol else f"{current_price:,.2f}"
        change_str = f"{change:+.4f} ({percent_change:+.2f}%)" if "C:" in ticker_symbol else f"{change:+.2f} ({percent_change:+.2f}%)"
        
        return f'<div class="market-card"><div class="card-header"><h3>{name}</h3><span class="price">{price_str}</span></div><div class="card-change {change_color_class}">{change_str}</div><div class="sparkline-chart">{chart_html}</div></div>'

    except Exception as e:
        return f"""<div class="market-card error-card">
                       <h3>{name}</h3>
                       <p>An error occurred for ticker '{ticker_symbol}':</p>
                       <pre><code>{e}</code></pre>
                   </div>"""

if api_key:
    client = RESTClient(api_key)
    uk_cards_html = "".join([create_market_card(client, symbol, name) for symbol, name in uk_tickers.items()])
    display(HTML(f'<div class="market-grid">{uk_cards_html}</div>'))
else:
    display(HTML("<p><i>Polygon.io API key not found. Please create a .env file in your project root with your key.</i></p>"))

## US

In [2]:
#| echo: false
# This python chunk generates the US market cards

# --- Tickers for the US dashboard (Polygon format) ---
us_tickers = {
    "AAPL": "APPLE",
    "MSFT": "MICROSOFT",
    "GOOG": "GOOGLE",
    "TSLA": "TESLA",
    "C:EURUSD": "EUR/USD",
    "C:USDJPY": "USD/JPY"
}

if api_key:
    client = RESTClient(api_key)
    us_cards_html = "".join([create_market_card(client, symbol, name) for symbol, name in us_tickers.items()])
    display(HTML(f'<div class="market-grid">{us_cards_html}</div>'))

## Europe

In [3]:
#| echo: false
# This python chunk generates the Europe market cards

# --- Tickers for the Europe dashboard (Polygon format) ---
europe_tickers = {
    "UNA": "UNILEVER",
    "VOW3": "VOLKSWAGEN",
    "C:EURCHF": "EUR/CHF",
    "C:EURNZD": "EUR/NZD",
    "NESN": "NESTLE",
    "C:EURGBP": "EUR/GBP"
}

if api_key:
    client = RESTClient(api_key)
    europe_cards_html = "".join([create_market_card(client, symbol, name) for symbol, name in europe_tickers.items()])
    display(HTML(f'<div class="market-grid">{europe_cards_html}</div>'))

## Asia

In [4]:
#| echo: false
# This python chunk generates the Asia market cards

# --- Tickers for the Asia dashboard (Polygon format) ---
asia_tickers = {
    "7203": "TOYOTA",
    "700": "TENCENT",
    "C:USDCNY": "USD/CNY",
    "BABA": "ALIBABA",
    "C:USDINR": "USD/INR",
    "X:BTCUSD": "BITCOIN"
}

if api_key:
    client = RESTClient(api_key)
    asia_cards_html = "".join([create_market_card(client, symbol, name) for symbol, name in asia_tickers.items()])
    display(HTML(f'<div class="market-grid">{asia_cards_html}</div>'))

:::